# ERG DATASET

Attempt to apply trained model on testset

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series, Timestamp
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator
import os
from numpy import load

In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.erg_wind import ErgFormatter

In [ ]:
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer
import libs.utils as utils

In [ ]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
# Tensorflow setup
default_keras_session: Session = tf1.keras.backend.get_session()
tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

In [ ]:
file_path: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\erg_wind\data\erg_7farms_final.csv'

In [ ]:
raw_data: DataFrame = pd.read_csv(file_path)

In [ ]:
raw_data.head()

In [ ]:
raw_data['time'] = raw_data['time'].astype('datetime64[s]')

In [ ]:
config = ExperimentConfig('erg_wind', r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs')

In [ ]:
formatter: ErgFormatter = config.make_data_formatter()

In [ ]:
data_csv_path: str = config.data_csv_path

# SPLIT DATA

In [ ]:
train, valid, test = formatter.split_data(raw_data)

In [ ]:
column_definitions = formatter.get_column_definition()

In [ ]:
train_samples, valid_samples = formatter.get_num_samples_for_calibration()

In [ ]:
# Sets up default params
fixed_params: Dict = formatter.get_experiment_params()
params: Dict = formatter.get_default_model_params()
params["model_folder"]: str = os.path.join(config.model_folder, "fixed")
model_folder = os.path.join(config.model_folder, "fixed")

In [ ]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)

In [ ]:
model_folder: str = opt_manager.hyperparam_folder

In [ ]:
test.head()

WE'RE EXPECTING THE MODEL TO BE FED INPUTS WHICH ARE STARTING FROM THE FOLLOWING TIMESTAMPS

In [ ]:
test.iloc[168:178, [0,1,25]]

# PREDICTION

In [ ]:
print("*** Running tests ***")
tf1.reset_default_graph()
with tf.Graph().as_default(), tf1.Session(config=tf_config) as sess:
    tf1.keras.backend.set_session(sess)
    params: Dict = opt_manager.get_next_parameters()
    params['exp_name'] = 'erg_wind'
    params['data_folder'] = os.path.abspath(os.path.join(data_csv_path, os.pardir))
    model = TemporalFusionTransformer(params, use_cudnn=False)
    params.pop('exp_name', None)
    params.pop('data_folder', None)
    # load model
    model.load(opt_manager.hyperparam_folder, use_keras_loadings=True)
    
#     print("Computing best validation loss")
#     val_loss: Series = model.evaluate(valid)
        
    print("Computing test loss")
    output_map: Dict = model.predict(test, return_targets=True)
    print(f"Output map returned a dict with keys {output_map.get('p50').shape}")
    targets: DataFrame = formatter.format_predictions(output_map["targets"])
    p50_forecast: DataFrame = formatter.format_predictions(output_map["p50"])
    p90_forecast: DataFrame = formatter.format_predictions(output_map["p90"])
        
    # save all
    print("saving predictions and targets")
    targets.to_csv(os.path.join(opt_manager.hyperparam_folder, "targets.csv"), index=False)
    p50_forecast.to_csv(os.path.join(opt_manager.hyperparam_folder, "p50.csv"), index=False)
    p90_forecast.to_csv(os.path.join(opt_manager.hyperparam_folder, "p90.csv"), index=False)
        
    def extract_numerical_data(data: DataFrame) -> DataFrame:
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p50_loss = utils.numpy_normalised_quantile_loss(
            extract_numerical_data(targets), extract_numerical_data(p50_forecast),
            0.5)
    p90_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p90_forecast),
        0.9)

    tf1.keras.backend.set_session(default_keras_session)

print()
print("Normalised Quantile Loss for Test Data: P50={}, P90={}".format(
    p50_loss.mean(), p90_loss.mean()))

In [ ]:
# test.columns

In [ ]:
# test.iloc[192:202, [0,20,21,24,23,2,3,4,5,6,7,8,9,10,11,12,13,14,15]]

# UNDERSTANDING SAVED PREDICTIONS

START BY CONVERTING THE FORECAST TIME INTO PROPER DATETIME FORMAT

In [ ]:
p50_forecast: DataFrame = pd.read_csv(os.path.join(opt_manager.hyperparam_folder, "p50.csv"))
p90_forecast: DataFrame = pd.read_csv(os.path.join(opt_manager.hyperparam_folder, "p90.csv"))
targets: DataFrame = pd.read_csv(os.path.join(opt_manager.hyperparam_folder, "targets.csv"))

In [ ]:
p90_forecast.head()

WE CAN'T FETCH DATES BECAUSE WE CONFIGURED THE TIME INDEX WITH THE "hours_from_start" WHICH GOT SCALED DURING PRE-TRAINING.
WE SHOULD RETRY USING THE DATETIME FORMATTED COLUMN AS TIME INDEX.